In [1]:
import pandas as pd
import json

In [2]:
import sys
sys.path.insert(0,'/home/madan/Desktop/joswin_bck/toPendrive/works/analytics_work_bench/nlp-intelligence/analytics_workbench' )

In [3]:
from analytics_workbench.process_text import ProcessText
from analytics_workbench.unsupervised_learning import Unsupervised

In [4]:
import psycopg2
database='crawler_service_test'
user='postgres'
password='postgres'
host='192.168.3.8'

con = psycopg2.connect(database=database, user=user,password=password,host=host)
cursor = con.cursor()

In [5]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in xrange(0, len(seq), size))

In [6]:
def load_data_for_text_processing(cursor,urls,batch_size=2000):
    for batch_urls in chunker(urls,batch_size):
        query = "select all_page_text from (select distinct on (domain) domain,all_page_text from "\
                    "webpage_texts where domain in %s)x"
        cursor.execute(query, (tuple(batch_urls),))
        # batch = cursor.fetchall()
        batch = []
        res = cursor.fetchone()
        while res:
            tt = json.loads(res[0])
            url_text = ''
            for k in tt.keys():
                if 'text_from_meta_' in k or 'text_from_tags_' in k or 'text_from_urls_' in k :#or 'text_from_urls' in k:
                    url_text = url_text + ' '+tt[k]
            batch.append((url_text,))
            res = cursor.fetchone()
        if not 1.0*len(batch)/batch_size > 0.8:
            continue
        if len(batch[0]) > 1:
            yield pd.DataFrame(batch).fillna('').apply(lambda x: '. '.join(x),1)
        else:
            batch = [i[0] for i in batch]
            yield pd.Series(batch).fillna('')

In [7]:
def get_data_matrix_iter(text_processor,cursor,ecom_urls,batch_size=3500):
    for inp_series in load_data_for_text_processing(cursor,ecom_urls,batch_size):
        yield text_processor.get_matrix_test(inp_series)[0]

In [11]:
query = 'select distinct domain from webpage_texts'
cursor.execute(query)
urls = cursor.fetchall()
urls = [i[0] for i in urls]

In [12]:
len(urls)

10347

In [11]:
data_yielder = load_data_for_text_processing(cursor,urls,batch_size=620)

In [12]:
text_processor = ProcessText()
text_processor.generate_vectorizer_iter_list(data_yielder,vectorizer_type='Count',synonym_loc=None,
                                stem_type=False,phrase_generation=False,
            stop_words_loc='/home/madan/Desktop/joswin_bck/toPendrive/works/company_classification/nlp-intelligence/stopwords.txt',
                lower=True,n_gram_range=(1,2),max_df=0.9,min_df=0.01)

In [16]:
len(text_processor.vocabulary)

29228

In [5]:
import dill

In [18]:
with open('website_vectorizer_march_14.pkl','w') as f:
    dill.dump(text_processor,f)

In [6]:
with open('website_vectorizer_march_14.pkl','r') as f:
    text_processor = dill.load(f)

In [13]:
matrix_iter = get_data_matrix_iter(text_processor,cursor,urls,188)
matrix_list = [i for i in matrix_iter]

In [14]:
from scipy.sparse import hstack,vstack
final_matrix = vstack(matrix_list)

In [21]:
final_matrix.shape

(10340, 29228)

In [24]:
with open('website_dtm_march_14.pkl','w') as f:
    dill.dump(final_matrix,f)

In [7]:
with open('website_dtm_march_14.pkl','r') as f:
    final_matrix = dill.load(f)

In [8]:
unsup_obj = Unsupervised()
model,mat_transform = unsup_obj.generate_topics(final_matrix,text_processor.vocabulary,'lda',10)

Topic #0:

product*174328.06, details*128135.79, full*120570.83, full product*112226.09, product details*111434.68, size*83752.30, made*82568.97, shop*72641.99, available*69473.75, contact*63427.90

arising related*0.10, thank visiting*0.10, severability event*0.10, platforms yotpo*0.10, made finest*0.10, information current*0.10, holidays processed*0.10, issue file*0.10, sign newsletter*0.10, encrypted payment*0.10


Topic #1:

necklaces*329888.53, leather*313384.11, necklace*221671.74, pendants*171063.41, gold*167314.08, cords*162526.76, jewelry*145883.85, chains*142923.19, earrings*123882.71, silver*104941.27

affect operation*0.10, copy characters*0.10, maple syrup*0.10, able see*0.10, sign different*0.10, dns disable*0.10, stored shopify*0.10, block lists*0.10, sign learn*0.10, work corrupt*0.10


Topic #2:

accessories*91978.48, home*63236.00, products*59028.13, furniture*51161.03, storage*48827.04, outdoor*32862.47, lighting*32607.26, contact*32282.83, equipment*31908.32, wall*3

In [9]:
unsup_obj = Unsupervised()
model,mat_transform = unsup_obj.generate_topics(final_matrix,text_processor.vocabulary,'nmf',10)

Topic #0:

necklaces*282.02, leather*281.89, collars*281.45, cords*281.37, pendants*161.89, necklace*149.97, chains*144.20, charms*4.84, gold*4.66, family*3.00

neckties*0.00, dragons*0.00, preferably*0.00, best allow*0.00, website uses*0.00, organizer*0.00, art piece*0.00, practices sites*0.00, used one*0.00, rating review*0.00


Topic #1:

true*159.25, option*156.47, inventory*156.11, sofa*138.55, price*119.71, title*108.07, shopify*102.19, products*101.85, cdn*101.59, files*101.53

expands*0.00, zip hoodie*0.00, arrives*0.00, ensure receive*0.00, long sleeved*0.00, bespoke*0.00, information reliance*0.00, pk*0.00, pj*0.00, pf*0.00


Topic #2:

dresses*300.38, size*107.17, prom*106.68, plus*78.83, plus size*78.72, long*62.12, mother*49.01, cheap*44.36, dress*43.75, formal*40.69

jewelry shop*0.00, waking*0.00, barbara*0.00, vitamin vitamin*0.00, miami fl*0.00, camcorders*0.00, even google*0.00, osborne*0.00, free warrant*0.00, red hook*0.00


Topic #3:

kings*264.50, product*73.14, g

In [16]:
with open('website_topic_model_lda50_march_15.pkl','w') as f:
    dill.dump(unsup_obj,f)

In [23]:
unsup_obj = Unsupervised()
unsup_obj.generate_clusters(final_matrix,text_processor.vocabulary,10)

No of items in each cluster
0    10330
4        2
7        1
6        1
5        1
3        1
2        1
9        1
1        1
8        1
dtype: int64
Top terms per cluster:
()
Cluster 0: shop, new, contact, product, home, cart, shipping, accessories, quick, sale
Cluster 1: necklaces, leather, collars, cords, pendants, necklace, chains, charms, gold, family
Cluster 2: title, inventory, option, price, attr, false, true, class, compare, list
Cluster 3: true, option, inventory, sofa, price, title, shopify, products, files, cdn
Cluster 4: price, option, inventory, title, true, shopify, black, compare, image, available
Cluster 5: dresses, prom, size, plus size, plus, long, mother, cheap, dress, formal
Cluster 6: black, onyx, yoga, black onyx, true, background, title, beaded, option, inventory
Cluster 7: red, bull, tee, racing, new, series, signature, york, new york, mitchell
Cluster 8: kings, galaxy, product, jerseys, th, th anniversary, anniversary, full, details, full product
Cluster 9: m

(KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
     n_clusters=10, n_init=10, n_jobs=1, precompute_distances='auto',
     random_state=None, tol=0.0001, verbose=0),
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

#### search option present in page or not

In [88]:
from bs4 import BeautifulSoup
query = "select domain,page_sources ->> 'base_url_source' from webpage_texts offset 70 limit 10"
cursor.execute(query)
res = cursor.fetchone()
while res:
    domain = res[0]
    soup = BeautifulSoup(res[1],'html.parser')
    forms = soup.find_all('form')
    search_present = False
    if forms:
        for form in forms:
            if 'search' in form.get('class','None') or 'search' in form.get('action','None') or \
                        'search' in form.get('role','None'):
                search_present=True
                break
    if not search_present:
        links_in_page = soup.find_all('a')
        if links_in_page:
            for link in links_in_page:
                if not(link.get('href','None').startswith('/') or domain in link.get('href','None')):
                    continue
                if 'search' in link.get('href') or 'search' in link.get_text('None'):
                    search_present = True
                    break
    print(domain,search_present)
    res = cursor.fetchone()

('shopyatra.com', True)
('beyoumultiwear.com', True)
('vossar.com', True)
('mocp.org', True)
('chicagokernel.com', True)
('allchicago247.com', True)
('studioeast6.com', True)
('floralexhibits.com', False)
('silvermoonvintage.com', True)
('mmdstyleboutique.com', True)


#### subscribe option present in page

In [81]:
from bs4 import BeautifulSoup
query = "select domain,page_sources ->> 'base_url_source' from webpage_texts offset 30 limit 10"
cursor.execute(query)
res = cursor.fetchone()
while res:
    domain = res[0]
    soup = BeautifulSoup(res[1],'html.parser')
    links_in_page = soup.find_all('a')
    subscr_present = False
    if links_in_page:
        for link in links_in_page:
            if not(link.get('href','None').startswith('/') or domain in link.get('href','None')):
                continue
            if 'subscrib' in link.get_text('None').lower():
                subscr_present = True
                break
    if not subscr_present:
        buttons = soup.find_all('button')
        if buttons:
            for button in buttons:
                if 'subscrib' in button.get_text('None').lower():
                    subscr_present = True
                    break
    print(domain,subscr_present)
    res = cursor.fetchone()

('theparaders.com', True)
('iheartkeenwah.com', False)
('ilfxm.com', True)
('givesugar.com', False)
('cameraambassador.com', True)
('shopfixture.com', False)
('salveobrands.com', False)
('nocs.se', False)
('theclassicgem.com', False)
('stranddesign.org', False)


#### finding shopify powered websites by looking for shopify link in the website

#### check if website is valid

In [86]:
import re
query = "select domain,page_sources ->> 'base_url_source' from webpage_texts offset 30 limit 10"
cursor.execute(query)
res = cursor.fetchone()
while res:
    domain = res[0]
    soup = BeautifulSoup(res[1],'html.parser')
    valid = True
    base_page_text = soup.get_text('None')
    if re.search('server not found|this shop is currently unavailable',base_page_text,re.I) or \
            re.search('Something went wrong',soup.title.get_text('None'),re.I) :
            valid = False
    print(domain,valid)
    res = cursor.fetchone()

('theparaders.com', True)
('iheartkeenwah.com', True)
('ilfxm.com', True)
('givesugar.com', True)
('cameraambassador.com', True)
('shopfixture.com', True)
('salveobrands.com', False)
('nocs.se', True)
('theclassicgem.com', True)
('stranddesign.org', True)


In [87]:
query = "select domain,page_sources ->> 'base_url_source' from webpage_texts offset 70 limit 10"
cursor.execute(query)
res = cursor.fetchone()
while res:
    domain = res[0]
    soup = BeautifulSoup(res[1],'html.parser')
    valid = True
    base_page_text = soup.get_text('None')
    if re.search('server not found|this shop is currently unavailable',base_page_text,re.I) or \
            re.search('Something went wrong',soup.title.get_text('None'),re.I) :
            valid = False
    print(domain,valid)
    res = cursor.fetchone()

('shopyatra.com', True)
('beyoumultiwear.com', True)
('vossar.com', True)
('mocp.org', True)
('chicagokernel.com', True)
('allchicago247.com', True)
('studioeast6.com', True)
('floralexhibits.com', True)
('silvermoonvintage.com', True)
('mmdstyleboutique.com', True)


In [154]:
query = "select misc_details -> 'shipping_providers' ship1,misc_details -> 'shipping_present_in_page' ship2,"\
        "misc_details -> 'seperate_shipping_page_present' ship3,misc_details -> 'mobile_app_present' mobile_app,"\
        "misc_details -> 'cart_present' cart,misc_details -> 'login_signup_present' login,"\
        "misc_details -> 'all_urls_in_website' from webpage_texts where domain = 'studioeast6.com'"
cursor.execute(query)
res = cursor.fetchall()

In [155]:
tmp = eval(res[0][6])

In [156]:
tmp1 = pd.DataFrame(tmp,columns=['url','url_text'])
tmp1['url'] = tmp1['url'].fillna('').apply(lambda x:x.strip())
tmp1['url_text'] = tmp1['url_text'].fillna('').apply(lambda x:x.strip())
tmp1 = tmp1.drop_duplicates()

In [157]:
tmp1['url_len'] = tmp1['url'].fillna('').apply(lambda x: len(x.strip()))
tmp1['url_text_len'] = tmp1['url_text'].fillna('').apply(lambda x: len(x.strip()))

In [164]:
list(tmp1.sort_values('url_text_len',ascending=False)['url'])[:40]

['http://www.studioeast6.com/pages/designers-we-want-you',
 'http://www.studioeast6.com#2',
 'http://www.studioeast6.com#4',
 'http://www.studioeast6.com#7',
 'http://www.studioeast6.com#6',
 'http://www.studioeast6.com/contact-us',
 'http://www.studioeast6.com#5',
 'http://www.studioeast6.com#8',
 'http://www.studioeast6.com#3',
 'https://www.studioeast6.com/pages/bridal-party',
 'http://www.studioeast6.com/pages/bridal-party',
 'http://www.studioeast6.com/pages/location-and-store-hours',
 'http://www.studioeast6.com/cart',
 'https://www.studioeast6.com/pages/groom',
 'http://www.studioeast6.com#1',
 'http://www.studioeast6.com/pages/terms-and-conditions',
 'http://www.studioeast6.com/pages/delivery-and-returns',
 'http://www.studioeast6.com/collections/shop-our-instagram',
 'https://www.studioeast6.com/pages/bridal',
 'https://www.studio-east6.maharaniweddings.com/',
 'http://www.studioeast6.com/collections/ready-to-ship-anarkalis',
 'http://www.studioeast6.com/collections/suit',
 'h

#### peer to peer

In [ ]:
regex = 'peer(.){0,3}(to|2)(.){0,3}peer' #only 28 in description